# Blackholes

Synthesizer has a collection of routines allowing us to also model the spectral energy distributions of black holes (i.e. AGN) and combine their emission with the emission of others galaxy components (i.e. stellar emission).

## Particle blackholes

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from unyt import Msun, yr, Angstrom

from synthesizer.particle.blackholes import BlackHoles
from synthesizer.particle.gas import Gas
from synthesizer.particle.galaxy import Galaxy

# Set a random number seed to ensure consistent results
np.random.seed(42)

### Creating a black hole

Before generating some simple observational quantities from physical properties we first need to create a `BlackHoles` object. This object can be found in `synthesizer/particle/blackholes.py`. 

Lets create an instance of `BlackHoles` containing 10 fake black holes. To do so we can provide a number of optional keyword arguments but for now lets just provide their masses, coordinates and accretion rates.

In [2]:
# Make fake properties
ms = np.full(10, 10**7)  # Msun
pos = np.random.normal(0, 1.5, (10, 3))  # cMpc
mdot = np.full(10, 100)  # Msun / yr

# And get the black holes object
bh = BlackHoles(masses=ms, coordinates=pos, accretion_rate=mdot)

#### Calculating black hole metallicity

If we want to calculate emission from the black hole we will need to know the metallicity of the emission regions surrounding the black hole. Above we could have passed an array of metallicities at instantiation but most of the time we will not know ahead of time what these values should be. Instead we can use the gas surrounding the black hole to calculate what this metallicity is. To do this we need to first create a `Galaxy` with both a `Gas` component and `BlackHoles`, again using fake data.

In [3]:
# Make fake gas properties
ngas = 200
ms = np.full(ngas, 10 ** 6.5)  # Msun
pos = np.random.normal(0, 1.5, (ngas, 3))  # cMpc
hsml = np.full(ngas, 0.75)  # cMpc
metals = np.full(ngas, 0.01)

# And make the gas object
gas = Gas(masses=ms, metallicities=metals, coordinates=pos, smoothing_lengths=hsml)

# And now create the galaxy 
galaxy = Galaxy(stars=None, gas=gas, black_holes=bh)

Now we have the galaxy we can use `galaxy.calculate_black_hole_metallicity()` to calculate the black holes' metallicity. This method will find all gas particles with smoothing lengths that intersect the black hole and calculate the mass weighted average of their metallicity. If a black hole does not find any gas neighbours then a default metallicity is set instead, by default this is solar metallicity (0.012) but can be overwritten by passing a new `default_metallicity` as shown below.

In [4]:
galaxy.calculate_black_hole_metallicity(default_metallicity=0.07)
print("Z_BH =", galaxy.black_holes.metallicities)

Z_BH = [0.01 0.07 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01]


### Calculating basic quantities

#### Bolometric luminosities

We might like to calculate the bolometric luminosity, 
$ L_{\rm \bullet, bol} = \epsilon_{r}\dot{M}_{\bullet}c^{2} $,
for each black hole. We can do so by using the interface built into `BlackHoles` (or indeed `Galaxy.black_holes`). Note that the efficency (`epsilon`) defaults to 0.1. 

In [5]:
Lbol = bh.calculate_bolometric_luminosity(epsilon=0.1)
# OR
Lbol = galaxy.black_holes.calculate_bolometric_luminosity(epsilon=0.1)
print(Lbol)

[5.66297517e+47 5.66297517e+47 5.66297517e+47 5.66297517e+47
 5.66297517e+47 5.66297517e+47 5.66297517e+47 5.66297517e+47
 5.66297517e+47 5.66297517e+47] erg/s


#### Big bump temperature

The default AGN model used by `synthesizer` is the one built into the `cloudy` photoionisation code:

$ T_{BB} = \left ( \frac{3c^{6}}{8\pi6^{3}\sigma_{SB}G^{2}} \frac{\dot{M}_{\bullet}}{M^{2}_{\bullet}} \right )^{1/4}  = 2.24 \times 10^{9} \left ( \frac{\dot{M}_{\bullet}}{M_{\odot}\,yr^{-1}} \right )^{1/4} \left ( \frac{M_{\bullet}}{M_{\odot}} \right )^{-1/2} $

This is calculated at instantiation of a`BlackHoles` object.

In [6]:
print(bh.bb_temperature)

[2240000. 2240000. 2240000. 2240000. 2240000. 2240000. 2240000. 2240000.
 2240000. 2240000.] K


## Parametric blackholes

## SED models

Like with modelling the SEDs of stellar populations, most of `synthesizer`'s blackhole SED modelling is enabled via grids pre-computed via the `cloudy` photoionisation code.

#### Feltre et al. (2016) model

One (slight) exception to this is the incident SED model used by Feltre et al. (2016) to model the NLRs of AGN. This is actually used by `synthesizer` to generate the incident SED for the creation of some grids but we can also explore the incident spectrum without loading the relevant grid. This model is a simple series of power-laws.

This model can be found in the `generate_grids` module and used as follows:

```
bh = blackholes.Feltre16() # initialise an instance of the Feltre16 class
alpha = -1
lam = np.arange(10, 1000000, 1) * Angstrom # define a wavelength grid
lnu = bh.incident(lam, alpha)
plt.loglog(lam, lnu)
```